In [36]:
import pandas as pd
import pyodbc
import sqlite3
from datetime import datetime

In [40]:
DB = {'servername': r'DESKTOP-I3QFF0F\SQLEXPRESS', 'database': 'DWH'}
now = datetime.now()
sqlite_connectie = sqlite3.connect('DWH')
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor


current_time= now.strftime("%d/%m/%Y %H:%M:%S")
current_time

'18/03/2024 15:14:06'

In [ ]:
staff_id = 15
satisfaction_data = {
    'SATISFACTION_YEAR_number': [2015],
    'SATISFACTION_STAFF_id': [staff_id],
    'SATISFACTION_TYPE_id': [1],
    'SATISFACTION_TYPE_description': ["goed"],
    'SALES_FSK': [pd.read_sql_query("select MAX(SALES_SK) from SALES where SALES.SALES_STAFF_id = " + str(staff_id), export_conn)],
    'Changedate': [current_time]
}

#max_sales_sk = (pd.read_sql_query('SELECT MAX(SALES_SK) FROM SALES', export_conn) + 1).iloc[0, 0]
sales_data = { 
    'SALES_SK':[changeSK(["SALES", "SATISFACTION"], "SALES")],
    'SALES_STAFF_id':[15],
    'SALES_FIRST_name':["Tim"],
    'SALES_LAST_name':["Visser"],
    'SALES_EXTENSION_number':[25],
    'SALES_EMAIL_name':["@mail.com"],
    'SALES_MANAGER_id':[11],
    'SALES_BRANCH_id':[3],
    'SALES_ADDRESS1_name':["straat3"],
    'SALES_ADDRESS2_name':["straat4"],
    'SALES_POSITION_name':["boss"],
    'SALES_WORK_PHONE_number':[12345],
    'SALES_FAX_number':[12],
    'SALES_HIRED_date' : ["5/12/2024"],
    'SALES_COUNTRY_id' : [3],
    'SALES_REGION_name' : ["W"],
    'SALES_CITY_name' : ["Utrecht"],
    'SALES_POSTAL_ZONE_name' : ["2325KL"],
    'SALES_FSK' : [],
    'Changedate': [current_time]
}

dbobject = pd.DataFrame(satisfaction_data)
data = pd.read_sql_query('SELECT * FROM SATISFACTION', sqlite_connectie)
dbobject2 = pd.DataFrame(sales_data)
pd.read_sql_query('SELECT * FROM SALES', sqlite_connectie)



In [ ]:
def changeSK(tables, table):
    SK = (pd.read_sql_query('SELECT MAX(' + table + '_SK) FROM ' + table, export_conn) + 1).iloc[0, 0]
    #pd.read_sql_query("select * from " + table + " join ")
    for i in tables:
        query = pd.read_sql_query("SELECT "+ tables[i] + "_SFK FROM "+ tables[i] + " WHERE ", export_conn)
        query[tables[i] + '_SFK'].replace(SK, inplace=True)
        
        for row in query.itertuples():
            query_change = f"UPDATE {tables[i]} SET {tables[i]}_SFK = {SK} WHERE {tables[i]}_SFK = {row[1]}"
            export_cursor.execute(query_change)
    return SK



In [ ]:
for index, row in dbobject.iterrows():
    query_satisfaction = f"INSERT INTO SATISFACTION VALUES ({row['SATISFACTION_YEAR_number']},'{row['SATISFACTION_STAFF_id']}', '{row['SATISFACTION_TYPE_id']}', '{row['SATISFACTION_TYPE_description']}', '{row['SALES_FSK']}', '{row['Changedate']}')"
export_cursor.execute(query_satisfaction)
for index, row in dbobject2.iterrows():
    query_sales = f"INSERT INTO SALES VALUES ({row['SALES_SK']},'{row['SALES_STAFF_id']}', '{row['SALES_FIRST_name']}', '{row['SALES_LAST_name']}', '{row['SALES_EXTENSION_number']}', '{row['SALES_EMAIL_name']}', '{row['SALES_MANAGER_id']}', '{row['SALES_BRANCH_id']}', '{row['SALES_ADDRESS1_name']}', '{row['SALES_ADDRESS2_name']}', '{row['SALES_POSITION_name']}', '{row['SALES_WORK_PHONE_number']}', '{row['SALES_FAX_number']}', '{row['SALES_HIRED_date']}', '{row['SALES_COUNTRY_id']}', '{row['SALES_REGION_name']}', '{row['SALES_CITY_name']}', '{row['SALES_POSTAL_ZONE_name']}', '{row['SALES_FSK']}', '{row['Changedate']}')"
export_cursor.execute(query_sales)

export_conn.commit()
export_cursor.close()



In [ ]:
dbobject.to_sql('SATISFACTION', sqlite_connectie, if_exists='replace', index = False)
dbobject2.to_sql('SALES', sqlite_connectie, if_exists='replace', index = False)